In [1]:
import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time
import pymysql

In [2]:
conn = pymysql.connect(host='49.247.132.235', user='kbo_db', password='1234',
                       db='kbo_data_db', charset='utf8')

In [3]:
curs = conn.cursor(pymysql.cursors.DictCursor)

In [4]:
sql = '''
select player_id
,group_concat(player_year SEPARATOR ',') as year 
from kbo_pitcher_total_status
group by  player_id;
'''

In [5]:
curs.execute(sql)

183

In [6]:
rows = curs.fetchall()

In [7]:
play_info = {}
for row in rows:
    play_info[row['player_id']] = row['year'].split(',')
# play_info


In [32]:
def get_status(bs,num,table_name,year,player_id):
    try:
        tag_name = '#contents > div.sub-content > div.player_records > \
        div:nth-child({}) > table > tbody > tr'.format(int(num))
        trs = bs.select(tag_name)
        for tr in trs:
            data_list = []
            tds = tr.select('td')
            for td in tds:
                data_list.append(td.text)
            sql = '''
                insert into {} values({},'{}','{}','{}','{}','{}','{}','{}','{}','{}','{}',
                '{}','{}','{}','{}','{}','{}','{}','{}','{}');
                '''.format(table_name,player_id,year,data_list[0],data_list[1],data_list[2]
                           ,data_list[3],data_list[4],data_list[5],data_list[6],data_list[7]
                           ,data_list[8],data_list[9],data_list[10],data_list[11],data_list[12]
                           ,data_list[13],data_list[14],data_list[15],data_list[16],data_list[17])
            curs.execute(sql)
    except Exception as err:
        print(str(err))

In [33]:
# driver = webdriver.Chrome('E:/chromedriver_win32/chromedriver.exe')
driver = webdriver.Chrome('E:/BigData/BigData/driver/chromedriver.exe')
#선수와 선수의 출전 년도로 반복문 실행
tag_num = {
            5:'kbo_pitcher_detail_team'
            ,7:'kbo_pitcher_detail_place'
#             ,11:'kbo_pitcher_detail_day'
#             ,13:'kbo_pitcher_detail_home_and_away'
#             ,15:'kbo_pitcher_detail_night'
#             ,17:'kbo_pitcher_detail_half'
        }

print('='*30,'start','='*30)
start_time =  time.time()
for player_id,years in play_info.items():
    url = 'https://www.koreabaseball.com/Record/Player/PitcherDetail/Game.aspx?playerId={}'.format(player_id)
    driver.get(url)
    time.sleep(1)
#     해당 선수의 년도별 페이지 반복문 실행
    try:
        for year in years:
            select = Select(driver.find_element_by_id('cphContents_cphContents_cphContents_ddlYear'))
            if year != '2000':
                select.select_by_value(str(year))
                time.sleep(1)
                html = driver.page_source
                bs = BeautifulSoup(html,'html.parser')
        #         해당 상황별 데이터를 반복문으로 데이터를 가져와서 인서트 쿼리 실행
                for num,table_name in tag_num.items():
                    get_status(bs,num,table_name,year,player_id)
                time.sleep(1)
    except:
        print('{}선수 {} 년도 에러발성'.foramt(player_id))
        pass       
end_time = time.time()
total_time = end_time - start_time
print('실행 완료')
print(total_time)

============================== start ==============================

                    insert into kbo_pitcher_detail_team values(60140,'2010','KIA','2','9.00','0','0','0','0','-','11',
                    '2','2','0','4','0','1','2','2','0.286');
                    

                    insert into kbo_pitcher_detail_team values(60140,'2010','SK','5','4.00','0','0','0','0','-','41',
                    '9','11','1','2','0','8','8','4','0.289');
                    

                    insert into kbo_pitcher_detail_team values(60140,'2010','넥센','2','3.86','0','0','0','0','-','12',
                    '2 1/3','3','0','2','0','5','1','1','0.300');
                    

                    insert into kbo_pitcher_detail_team values(60140,'2010','두산','2','2.08','0','0','0','1','-','18',
                    '4 1/3','2','0','4','0','5','2','1','0.143');
                    

                    insert into kbo_pitcher_detail_team values(60140,'2010','롯데','3','37.80','0','1','0','0','0.0